In [1]:
#reference https://github.com/mokemokechicken/keras_BEGAN
#reference https://github.com/siddharthalodha/BEGAN_KERAS/blob/master/BEGAN_v1.ipynb    

In [76]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU , PReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.initializers import truncated_normal , constant , random_normal

from keras.optimizers import Adam , RMSprop

#残差块使用
from keras.layers import Add

from keras.datasets import mnist

#addin BEGAN
from keras.layers import Lambda #可以自己指定特定层 完成特定的功能 discriminator使用
from keras.layers import Concatenate #将discriminator的两个输出合并使用

In [79]:
import os

import matplotlib as plt
import numpy as np

import gc

from glob import glob

import keras.backend as K

from scipy.misc import imread , imsave

%matplotlib inline

In [62]:
#BEGAN使用64*64*3 图像
WIDTH = 64
HEIGHT = 64
CHANNEL = 3

SHAPE = (WIDTH , HEIGHT , CHANNEL)

LATENT_DIM = 64 #latent variable z sample from normal distribution

BATCH_SIZE = 1
EPOCHS = 10

PATH = '../dataset/CelebA/img_align_celeba/'

#生成多少个图像 长*宽
ROW = 2
COL = 2

#addin BEGAN
N_FILTERS = 128

In [63]:
#==============
IMAGES_PATH = glob(PATH+'*')

In [90]:
def load_image(batch_size = BATCH_SIZE , training = True):
    #随机在图片库中挑选
    image_path = np.random.choice(IMAGES_PATH , size=batch_size)
    
    images = []
    
    for an_image_path in image_path:
        img = imread(an_image_path , mode='RGB').astype(np.float)
        
        #尽管原图像不是指定的大小 下面将强制将图像resize
        img = scipy.misc.imresize(img , size=SHAPE) #resize到64*64*3尺寸
        
        #随机性地对训练样本进行 左右反转
        #BEGAN不进行左右反转
        #if training and np.random.random()<0.5:
        #    img = np.fliplr(img)
        
        images.append(img)
        
    images = np.array(images)/127.5 - 1
    
    return images

def write_image(epoch):
    #生成高分图像时 进行对比显示
    z = np.random.uniform(-1 , 1 , size=(ROW*COL , LATENT_DIM))
    images = Generator.predict(z)
    
    images = images*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    count=0
    
    axes[0][0].imshow(images[0])
    #axes[0][0].set_title('original high')
    axes[0][0].axis('off')

    axes[0][1].imshow(images[1])
    #axes[0][1].set_title('generated high')
    axes[0][1].axis('off')
    

    axes[1][0].imshow(images[2])
    #axes[1][0].set_title('original high')
    axes[1][0].axis('off')

    axes[1][1].imshow(images[3])
    #axes[1][1].set_title('generated high')
    axes[1][1].axis('off')
    
    fig.savefig('celeba_began/No.%d.png' % epoch)
    plt.pyplot.close()


In [6]:
#==============

In [65]:
def multi_conv2d(x , output_size , strides , n_layer):
    for _ in range(n_layer):
        x = Conv2D(output_size , kernel_size=(3,3) , strides=(1,1) , activation='elu' , padding='same')(x) #1步卷积 保证图像尺寸不变
    
    return Conv2D(output_size , kernel_size=(3,3) , strides=strides , activation='elu' , padding='same')(x) #最后的这个卷积的stride为2 进行降2倍采样


def multi_deconv2d(x , output_size , n_layer , upsample):
    for _ in range(n_layer):
        x = Conv2D(output_size , kernel_size=(3,3) , strides=(1,1) , activation='elu' , padding='same')(x)
    
    if upsample:
        x = UpSampling2D()(x)

    return x


In [66]:
def encoder():
    image = Input(shape=SHAPE)
    
    h = multi_conv2d(image , output_size=N_FILTERS , strides=(2,2) , n_layer=2)
    h = multi_conv2d(h , output_size=N_FILTERS*2 , strides=(2,2) , n_layer=2)
    h = multi_conv2d(h , output_size=N_FILTERS*3 , strides=(2,2) , n_layer=2)
    h = multi_conv2d(h , output_size=N_FILTERS*4 , strides=(1,1) , n_layer=2)
    
    h = Flatten()(h)
    
    feature = Dense(units=64 , activation='linear')(h)
    
    return Model(image , feature)
    
def decoder():
    feature = Input(shape=(64 , ))
    
    h = Dense(units=8*8*N_FILTERS , activation='linear')(feature)
    h = Reshape(target_shape=(8,8,N_FILTERS))(h)
    
    h = multi_deconv2d(h , output_size=N_FILTERS , n_layer=2 , upsample=True)
    h = multi_deconv2d(h , output_size=N_FILTERS , n_layer=2 , upsample=True)
    h = multi_deconv2d(h , output_size=N_FILTERS , n_layer=2 , upsample=True)
    h = multi_deconv2d(h , output_size=N_FILTERS , n_layer=2 , upsample=False)
    
    image = Conv2D(3 , kernel_size=(3,3) , strides=(1,1) , activation='linear' , padding='same')(h)
    
    return Model(feature , image)


In [67]:
def autoencoder():
    image = Input(shape=(SHAPE))
    
    Encoder = encoder()
    Decoder = decoder()
    
    feature = Encoder(image)
    image_hat = Decoder(feature)
    
    return Model(image , image_hat)

In [68]:
def discriminator():
    input_data = Input(shape=(HEIGHT , WIDTH , CHANNEL*2)) #因为discriminator的输入是两幅图像 所以输入的通道翻一倍
    
    real_image = Lambda(lambda x: x[:,:,:,:3] , output_shape=SHAPE)(input_data)
    generator_image = Lambda(lambda x: x[:,:,:,3:] , output_shape=SHAPE)(input_data)
    
    Autoencoder = autoencoder()
    
    real_image_hat = Autoencoder(real_image)
    generator_image_hat = Autoencoder(generator_image)

    output_data = Concatenate()([real_image_hat , generator_image_hat])

    return Model(input_data , output_data)

In [69]:
def generator():
    return decoder() #feature to image

In [84]:
class D_loss(object):
    __name__ = 'discriminator_loss'
    
    def __init__(self , init_k_var=0.0 , init_lambda_k = 0.001 , init_gamma=0.5):
        self.lambda_k = init_lambda_k
        self.gamma = init_gamma
        self.k_var = K.variable(init_k_var , dtype=K.floatx())
        
        self.m_global_var = K.variable(0.0 , dtype=K.floatx())
        self.loss_real_x_var = K.variable(0)
        self.loss_gene_x_var = K.variable(0)
        
        self.updates = []

    def __call__(self , y_true , y_pred):
        real_image_hat_true , generator_image_hat_true = y_true[:,:,:,:3] , y_true[:,:,:,3:]
        real_image_hat_pred ,generator_image_hat_pred = y_pred[:,:,:,:3] , y_true[:,:,:,3:]

        #下面的平均在batch维上还没有平均 所以得到的是1维张量
        loss_real_image_hat = K.mean(K.abs(real_image_hat_true-real_image_hat_pred) , axis=[1,2,3])
        loss_generator_image_hat = K.mean(K.abs(generator_image_hat_true-generator_image_hat_pred) , axis=[1,2,3])

        #paper equation
        discriminator_loss = loss_real_image_hat - self.k_var*loss_generator_image_hat

        mean_loss_real_image_hat = K.mean(loss_real_image_hat)
        mean_loss_generator_image_hat = K.mean(loss_generator_image_hat)
        #paper equation
        new_k = self.k_var + self.lambda_k*(self.gamma*mean_loss_real_image_hat-mean_loss_generator_image_hat)
        new_k = K.clip(new_k , 0 , 1)
        self.updates.append(K.update(self.k_var , new_k))
        
        m_global = mean_loss_real_image_hat + K.abs(self.gamma*mean_loss_real_image_hat-mean_loss_generator_image_hat)
        self.updates.append(K.update(self.m_global_var , m_global))
        
        self.updates.append(K.update(self.loss_real_x_var , mean_loss_real_image_hat))
        self.updates.append(K.update(self.loss_gene_x_var , mean_loss_generator_image_hat))
        
        return discriminator_loss

    @property
    def k(self):
        return K.get_value(self.k_var)
    
    @property
    def m_global(self):
        return K.get_value(self.m_global_var)
    
    @property
    def loss_real_x(self):
        return K.get_value(self.loss_real_x_var)
    
    @property
    def loss_gene_x(self):
        return K.get_value(self.loss_gene_x_var)

In [71]:
def build_generator_loss(Autoencoder):
    def generator_loss(y_true , y_pred):
        y_pred_dash = Autoencoder(y_pred)
        
        return K.mean(K.abs(y_pred - y_pred_dash) , axis=[1,2,3])
    
    return generator_loss

In [85]:
Autoencoder = autoencoder()
Generator = generator()
Discrimminator = discriminator()

In [86]:
loss_discriminator = D_loss()
Discrimminator.compile(optimizer=Adam(lr=0.0002 , beta_1=0.5) , loss=loss_discriminator)

In [87]:
Generator.compile(optimizer=Adam(lr=0.0002 , beta_1=0.5) , loss=build_generator_loss(Autoencoder))

In [92]:
for i in range(11):
    
    #训练discriminator
    input_x1 = load_image() #真实图像
    input_x2 = Generator.predict(np.random.uniform(-1,1, size=(BATCH_SIZE , LATENT_DIM)))
    input_x = np.concatenate((input_x1 , input_x2) , axis=-1)
    
    loss_d = Discrimminator.train_on_batch(input_x , input_x)

    #训练generator
    input_x3 = np.random.uniform(-1,1 , size=(BATCH_SIZE , LATENT_DIM))
    loss_g = Generator.train_on_batch(input_x3 , np.zeros_like(input_x2))
    
    print('epoch:%d loss_D:%f loss_G:%f' % (i , loss_d , loss_g))

    #if i % 50 == 0:
    write_image(i)
    
#write_image(999)


I:\Anaconda3\lib\site-packages\scipy\misc\pilutil.py:480: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if issubdtype(ts, int):
I:\Anaconda3\lib\site-packages\scipy\misc\pilutil.py:483: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


epoch:0 loss_D:0.624767 loss_G:0.030820
epoch:1 loss_D:0.395801 loss_G:0.030991
epoch:2 loss_D:0.541951 loss_G:0.025814
epoch:3 loss_D:0.585128 loss_G:0.025797
epoch:4 loss_D:0.561813 loss_G:0.024716
epoch:5 loss_D:0.378884 loss_G:0.024225
epoch:6 loss_D:0.514190 loss_G:0.020481
epoch:7 loss_D:0.927406 loss_G:0.018667
epoch:8 loss_D:0.477981 loss_G:0.018754
epoch:9 loss_D:0.577440 loss_G:0.019124
epoch:10 loss_D:0.402025 loss_G:0.017704


In [70]:
real_labels.shape

(64, 1)

In [ ]:
gc.collect()

In [ ]:
gc.collect()

In [2]:
VGG19(weights='imagenet')

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of cbe5617147190e668d6c5d5026f83318 so we will re-download the data.
574717952/574710816 [==============================] - 729s 1us/step
